In [ ]:
pip install -q yahoo_fin

In [4]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.6#egg=runml

In [5]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [6]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [7]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP',
           'BASE', 'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'HIMS','HUBS',
           'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'RMBS', 'SMCI', 'SMAR', 'SNAP', 'SNOW', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-6a', mod, True, loss=lossfn)[0]

In [ ]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

In [ ]:
findata.EPOCHS=200
later = [ ]
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-5b', mod, False)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [ ]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
10,IPO,RROnly,24.529832,0.962766,251.14,320.67,571.81,39.56,43.340000,0.10,9.255319
1,ARKK,RROnly,31.119823,0.920213,579.44,582.25,1161.69,43.32,47.980000,0.11,8.476789
2,ARKW,RROnly,38.526062,0.898936,674.75,617.32,1292.07,76.67,89.940002,0.17,8.394869
44,XME,RROnly,20.575078,0.914894,307.54,177.60,485.14,59.06,54.880001,-0.07,7.933131
33,XLB,RROnly,53.212842,0.936170,354.16,224.51,578.67,89.64,85.639999,-0.04,7.765957
32,XITK,RROnly,92.808587,0.898936,775.39,1097.26,1872.65,142.49,153.740005,0.08,7.726064
0,ARKF,RROnly,14.373508,0.856383,177.71,218.92,396.63,27.58,32.310001,0.17,7.719024
21,ONLN,RROnly,27.056857,0.888298,235.93,327.51,563.44,40.31,43.939999,0.09,7.641844
31,WDIV,RROnly,44.056150,0.941489,162.92,113.67,276.60,59.91,58.029999,-0.03,7.464539
15,IWO,RROnly,193.895751,0.936170,1032.89,1069.89,2102.77,260.22,267.140015,0.03,7.234043


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
tickers3 = [ 'AIRI', 'AEXAF', 'ALSMY', 'AMSC', 'AMSSY', 'ACHR',
            'APPS', 'AUGX', 'AUR',  'ATOM', 'AWRE', 'AXTI',
            'BKKT', 'BGSF', 'BLDE', 'BLNK',
            'CARS', 'CLOV', 'CRCT', 'CXM', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY',
            'GOGO', 'GRAB', 'GRPN', 'GTE', 'HIMX', 'HIVE',
            'INDI', 'INLX',  'INVZ', 'JOBY',  'KIND',
            'LAZR', 'LFMD', 'LUMN', 'MAX', 'MRDB', 'MTTR', 'MYTE',
            'NEPH', 'NNOX', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PGY', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RUM', 'RVYL',
            'SABR', 'SAVE', 'SFIX', 'SLDP', 'SLE', 'SMWB', 'SOUN', 'SST', 'STEM',
            'TDW', 'TRUP', 'TELL',
            'VTEX', 'YEXT', 'WKME']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-6b', mod, True)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3[(df3.Gain>0)].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
7,AUGX,RROnly,0.900917,0.920000,29.16,46.83,75.99,0.80,1.250000,0.55,5.266310
6,APPS,RROnly,2.952001,0.920213,421.94,498.77,920.71,1.46,2.060000,0.41,5.218413
19,CXM,RROnly,7.950363,0.942029,62.87,110.06,172.93,8.73,10.210000,0.17,5.200341
49,ONDS,RROnly,0.530255,0.903614,45.13,66.79,111.92,0.69,0.910000,0.32,5.014174
23,DMTK,RROnly,1.040155,0.835106,238.36,234.68,473.04,0.11,0.540000,3.70,4.867825
61,RVYL,RROnly,3.430919,0.856383,389.76,908.20,1297.96,1.46,2.430000,0.67,4.823193
47,NNOX,RROnly,6.048989,0.861111,253.33,343.85,597.18,6.98,9.920000,0.42,4.734672
70,STEM,RROnly,1.874044,0.873563,178.44,293.66,472.10,1.13,1.470000,0.30,4.717151
68,SOUN,RROnly,1.145403,0.906250,27.92,34.40,62.32,4.00,4.590000,0.15,4.705882
34,INDI,RROnly,5.036675,0.904255,89.63,87.46,177.09,6.38,7.280000,0.14,4.635258


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DELL', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG', 'LLY',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX',
            'TMO', 'TSLA', 'TXN', 'UNH', 'UPS', 'WMT']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-5b', mod4, False)[0]

In [9]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
2,ADBE,RROnly,282.879698,0.930851,3143.79,3120.10,6263.89,555.54,508.760010,-0.08,5.784574
11,CRM,RROnly,132.181681,0.904255,1213.46,1557.54,2771.00,257.10,225.580002,-0.12,5.661094
34,SBUX,RROnly,67.863903,0.893617,321.18,412.33,733.52,77.85,84.989998,0.09,5.200946
36,TSLA,RROnly,104.519260,0.856383,2280.15,2631.22,4911.37,197.88,176.429993,-0.11,4.811412
37,TXN,RROnly,117.817157,0.888298,740.12,571.89,1312.01,194.53,181.199997,-0.07,4.749240
28,NKE,RROnly,80.874272,0.781915,490.89,566.75,1057.65,75.37,92.989998,0.23,4.636910
21,INTC,RROnly,25.187652,0.835106,213.70,247.25,460.95,30.97,33.480000,0.08,3.903875
22,IBM,RROnly,87.926482,0.861702,521.59,341.11,862.71,172.95,162.580002,-0.06,3.850051
19,FDX,RROnly,145.108907,0.813830,936.71,864.73,1801.43,299.84,274.109985,-0.09,3.744512
30,NOW,RROnly,355.692425,0.824468,3122.51,2940.93,6063.44,786.67,725.390015,-0.08,3.694909


In [ ]:
findata.EPOCHS=200
tickers5 = ['ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX', 'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY',
            'EQNR', 'ERIC', 'ERJ','ESEA', 'EVKIY',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LIN', 'LYB', 'LVMUY',
            'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-6b', mod5, True)[0]


In [12]:
addAlloc(df5, 0.14)
df5[df5.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
24,FQVLF,RROnly,7.686009,0.920213,152.29,259.34,411.63,13.20,16.469999,0.25,5.934650
62,TKAMY,RROnly,4.158087,0.930851,73.63,70.06,143.69,4.38,5.060000,0.16,5.784574
2,ADYEY,RROnly,7.129250,0.909574,150.22,166.93,317.16,11.87,14.540000,0.22,5.674910
55,SE,RROnly,40.423803,0.893617,1150.49,1558.02,2708.51,71.42,84.120003,0.18,5.200946
45,OTGLY,RROnly,4.353191,0.819149,55.78,93.30,149.07,8.60,10.740000,0.25,4.404255
3,BAK,RROnly,6.556585,0.909574,137.75,125.20,262.96,6.45,6.900000,0.07,3.913374
14,EADSF,RROnly,68.409674,0.898936,753.03,429.35,1182.38,137.01,147.839996,0.08,3.894377
48,PROSF,RROnly,28.452490,0.845745,370.11,527.37,897.48,37.15,41.970001,0.13,3.667875
57,STLA,RROnly,6.901286,0.909574,116.19,69.37,185.56,19.85,20.969999,0.06,3.482776
61,THQQF,RROnly,1.919248,0.739362,88.38,89.16,177.55,2.22,2.760000,0.24,3.109169


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'FLN', 'FM',
           'ECH', 'EPHE', 'EWA', 'EWI', 'EWJ',
           'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
           'IDX', 'ILF', 'INDA', 'KEMQ', 'THD', 'TUR', 'VNM', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-3b', mod6, True)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
14,EWZ,RROnly,20.870570,0.946809,163.56,134.32,297.88,29.60,31.480000,0.06,8.581560
11,EWP,RROnly,18.915227,0.914894,109.69,65.86,175.56,33.66,29.010000,-0.14,8.541033
6,EWA,RROnly,16.264419,0.936170,94.19,62.48,156.67,24.65,22.930000,-0.07,8.449848
9,EWC,RROnly,23.547080,0.920213,110.88,82.88,193.76,38.26,34.590000,-0.10,8.404255
22,XCEM,RROnly,20.731747,0.930851,109.14,71.15,180.30,30.51,28.850000,-0.05,7.925532
2,FLN,RROnly,12.479329,0.930851,88.30,48.80,137.11,19.31,18.270000,-0.05,7.925532
20,TUR,RROnly,16.397902,0.888298,185.40,124.43,309.83,42.28,37.439999,-0.11,7.867505
7,EWI,RROnly,20.596518,0.877660,122.11,76.85,198.96,38.83,33.860001,-0.13,7.835516
1,EWD,RROnly,26.322633,0.925532,134.10,155.81,289.91,41.93,39.959999,-0.05,7.765957
19,THD,RROnly,56.310138,0.909574,217.91,250.43,468.35,57.55,60.330002,0.05,7.287234


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
            'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SBLK', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-5a', mod7, False)[0]

In [ ]:
addAlloc(df7, 0.10, 1)
df7[df7.Gain > 0].sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
54,KVUE,RROnly,17.879241,0.953488,10.19,30.07,40.26,18.39,19.459999,0.06,8.759690
9,BAX,RROnly,31.732772,0.904255,180.04,290.56,470.60,33.16,39.580002,0.19,8.538634
59,NTR,RROnly,29.110441,0.904255,451.17,288.11,739.28,55.41,64.650002,0.17,8.479349
5,AQN,RROnly,4.968251,0.936170,41.59,55.56,97.15,5.94,6.370000,0.07,8.449848
75,SAFE,RROnly,16.065079,0.888298,416.94,575.85,992.78,19.06,22.940001,0.20,8.324468
11,BGFV,RROnly,1.650961,0.882979,108.73,129.18,237.91,3.27,3.900000,0.19,8.213886
66,PETS,RROnly,4.463863,0.861702,77.42,123.24,200.66,4.34,5.500000,0.27,8.104807
67,PNR,RROnly,34.983571,0.914894,300.59,244.97,545.56,77.55,83.169998,0.07,7.933131
2,ALB,RROnly,77.683170,0.882979,1808.97,1708.83,3517.81,114.94,130.119995,0.13,7.929624
61,OLN,RROnly,10.142511,0.888298,325.81,313.54,639.35,50.07,55.500000,0.11,7.867505


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'FXY', 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-6a', mod8, True)[0]

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
4,SLV,RROnly,16.525718,0.925532,112.88,90.04,202.92,26.67,28.340000,0.06,8.014184
0,FXY,RROnly,59.183566,0.904255,84.37,165.38,249.75,59.00,63.480000,0.08,7.845745
7,TLT,RROnly,81.844570,0.898936,224.26,330.29,554.56,91.50,85.720001,-0.06,7.304965
1,GLD,RROnly,152.558409,0.914894,491.76,432.03,923.79,211.60,219.740005,0.04,7.021277
10,WEAT,RROnly,5.004930,0.819149,34.19,34.31,68.50,5.90,6.710000,0.14,6.899696
5,SOYB,RROnly,14.039792,0.872340,84.16,61.68,145.84,24.67,25.350000,0.03,4.468085
2,ISHG,RROnly,63.519917,0.920213,65.64,130.73,196.37,69.90,70.540001,0.01,1.223404
6,TIP,RROnly,100.435658,0.867021,98.49,104.66,203.14,106.05,104.790001,-0.01,0.000000
3,PDBC,RROnly,7.095595,0.840426,51.31,21.92,73.23,13.75,13.550000,-0.01,0.000000
8,UUP,RROnly,22.566834,0.856383,37.29,20.80,58.09,28.77,28.379999,-0.01,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'BCH-USD', 'DOGE-USD', 'DOT-USD',
            'ETH-USD', 'FIL-USD', 'ICP-USD', 'LINK-USD', 'LTC-USD',
            'MATIC-USD', 'SOL-USD', 'TON-USD', 'TRX-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-5a', mod9, False)[0]

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Error,Accuracy,Buy,Sell,Total,Last,Predicted,Gain,Alloc
6,ETH-USD,RROnly,1088.746444,0.867021,14249.41,22601.37,36850.78,3556.97,3078.899902,-0.13,5.170096
1,AVAX-USD,RROnly,11.517795,0.877660,659.78,599.52,1259.30,30.19,33.310001,0.10,5.045593
10,LTC-USD,RROnly,48.377828,0.840426,390.60,1120.19,1510.79,78.66,89.919998,0.14,4.863222
8,ICP-USD,RROnly,4.108900,0.760638,65.94,187.36,253.30,9.20,12.020000,0.31,4.660996
5,DOT-USD,RROnly,4.506689,0.877660,75.77,171.58,247.35,6.21,5.800000,-0.07,4.521277
9,LINK-USD,RROnly,5.840173,0.851064,137.61,149.19,286.80,14.89,13.760000,-0.08,4.217325
0,ADA-USD,RROnly,0.289958,0.813830,4.23,8.08,12.31,0.41,0.370000,-0.11,4.120613
15,XRP-USD,RROnly,0.329687,0.792553,3.63,5.69,9.32,0.48,0.530000,0.10,3.586626
2,BTC-USD,RROnly,17308.521045,0.797872,248096.25,180756.46,428852.71,66167.43,60342.859375,-0.09,3.453225
12,SOL-USD,RROnly,14.923197,0.781915,590.56,1183.59,1774.15,144.64,135.289993,-0.06,1.950355


In [13]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
gstock-6b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,221,999 bytes  received 35 bytes  6,444,068.00 bytes/sec
total size is 218,358,952  speedup is 67.77
/content/drive/MyDrive/colab/results/mcap-3b-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/mcap-3b-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/mcap-3b-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/mcap-3b-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/cetf-3b-tes